In [1]:
from pymongo import MongoClient
import pymongo
import pandas as pd
import numpy as np
import scipy as sp
import scipy.signal as sig
from ipyparallel import Client
import sys
import numbapro

In [2]:
client = MongoClient()
db = client.VentDyssynchrony_db
test_data = db.TestData_collection
breath_data = db.BreathData_collection
RN_data = db.RNData_collection
patient_data = db.PatientData_collection

In [ ]:
for items in ['P128', 'P129', 'P130']:
    print(items, breath_data.find({'patientID': items}).count())
    print(items, RN_data.find({'patientID': items}).count())

In [3]:
ipclient = Client()
print (ipclient.ids)
#ipview = ipclient[:]
ipview = ipclient.load_balanced_view()

[0, 1, 2, 3]


In [ ]:
#breath_data.ensure_index([('analysis', pymongo.ASCENDING)], name='Analysis')

In [ ]:
#breath_data.find({'analysis':{'$exists':1}}).count()

{'bye', 'hi'}

## Define Parallel Loop

In [59]:
@ipview.parallel(block=True)
def parallel_dyssynchrony_finder(patients):
    # import necessary dependencies
    from pymongo import MongoClient
    import pandas as pd
    import numpy as np
    import scipy as sp
    import scipy.signal as sig
    import sys
    import numbapro

    # open mongoClients
    client = MongoClient()
    db = client.VentDyssynchrony_db
    breath_data = db.BreathData_collection
    #breath_data = db.TestData_collection

    # define some basic variables
    unpack_col = []
    p_or_f = 'flow'
    n_max = np.array([0])
    n_min = np.array([0])
    breath_df = pd.DataFrame()
    
    # define subfunctions
    # unpacking raw dataframe
    def unpack(breath, unpack_col = unpack_col):
        for col in unpack_col:      
            key_names = set()
            keys = set()
            key = ''

            try:
                keys = breath[col].apply(lambda x: set(x.keys()) if isinstance(x, dict) else set())
                #if col == 'analysis':
                #    keys = keys.union(set(['ds', 'pds', 'ie', 'fl', 'pl', 'pvt']))
            except:
                print('first keys', sys.exc_info()[0])
                #if col == 'analysis':
                #    keys = set(['ds', 'pds', 'ie', 'fl', 'pl', 'pvt'])

            try:
                for items in keys:
                    key_names.update(items)
            except:
                print(sys.exc_info()[0])
                print('second keys', keys)

            try:
                for key in key_names:
                    breath[col +':'+ key] = breath[col].apply(lambda x: x[key] if (isinstance(x, dict) and (key in x)) else np.nan)
                breath.drop(col, inplace=True, axis=1)
            except:
                print(sys.exc_info()[0])
                print('last', key, key_names)

        return breath

    # makes df for analysis
    def make_analysis_df(row, p_or_f=p_or_f):
        analysis_df = pd.DataFrame({'flow': np.array(row['data_frame:Flow']),'paw':np.array(row['data_frame:Paw']), 'status':np.array(row['data_frame:Status']), 
                                    'time':np.array(row['data_frame:Time']), 'volume':np.array(row['data_frame:Volume']), 'breath_num':row['breath_number']})
        analysis_df.set_index('time', inplace=True, drop=False)
        analysis_df['status'] = analysis_df['status'].apply(lambda x: 1 if x>0 else 0)

        try:
            smoothed_array = sig.savgol_filter(analysis_df[p_or_f].values, window_length=7, polyorder=2)
            analysis_df['smooth'] =  pd.Series(smoothed_array, index=analysis_df.index)
        except:
            analysis_df['smooth'] =  analysis_df[p_or_f]
            print('make', sys.exc_info()[0])

        return analysis_df

    #find max and min function
    def max_min(concave_index, analysis_df, max_or_min):
        count = 0
        end = []
        start = [concave_index[0]]
        while count < concave_index.size -3:
            if concave_index[count] + 1 < concave_index[count+1] and concave_index[count] + 2 != concave_index[count+2] and concave_index[count] + 3 != concave_index[count+3]:
                start.append(concave_index[count+1])
                end.append(concave_index[count])
            count = count + 1
        start_end = list(zip(start, end))

        n_index = np.array([])
        for items in start_end:
            if items[1]-items[0] >= 3:
                if max_or_min == 'max':
                    n_index = np.append(n_index, np.array(np.argmax(analysis_df['smooth'].values[items[0]:items[1]])+items[0]))
                elif max_or_min == 'min':
                    n_index = np.append(n_index, np.array(np.argmin(analysis_df['smooth'].values[items[0]:items[1]])+items[0]))
        return n_index

    # find post_ds breaths function
    def find_post_ds(row):
        breath = breath_data.find_one({'file_name': row['file_name'], 'breath_number':row['breath_number']+1}, {'_id':1, 'characteristics.max_vol':1})

        if isinstance(breath, type(None)):
            print(breath, row['_id'], row['file_name'], row['breath_number'])
            return np.nan
        else:

            max_vol = breath['characteristics']['max_vol']
            ds_vol = max_vol + row['characteristics:end_insp_vol']
            try:   
                breath_data.update_one({'_id':row['_id']}, {'$set':{'characteristics.ds_vol':ds_vol}})
            except:
                print('vol_update',sys.exc_info()[0])
            try:
                breath_data.update_one({'file_name': row['file_name'], 'breath_number':row['breath_number']+1}, {'$set':{'analysis.pds':True}})
            except pymongo.errors.WriteError as e:
                print(e)
                print(breath_data.find_one({'file_name': row['file_name'], 'breath_number':row['breath_number']+1}, {'analysis.pds':True}))
                print('pds_update', sys.exc_info()[0])
            return ds_vol

    #find ie breaths function
    def find_ie(row, p_or_f=p_or_f):
        if row['characteristics:max_vol'] < 100:
            return True

        if row['characteristics:exp_time'] < 0.75:
            return False

        analysis_df = make_analysis_df(row,  p_or_f=p_or_f)

        n_max = np.array([])
        n_min = np.array([])

        exp_start = analysis_df[analysis_df.status == 1].shape[0]
        exp_end = analysis_df.shape[0]
        search_start = int((exp_end-exp_start)*0.3 + exp_start)
        search_delta = exp_end - search_start

        analysis_df['diff1'] = analysis_df['smooth'].diff()
        analysis_df['diff2'] = analysis_df['diff1'].diff()

        concave_down_index = np.ravel(np.argwhere(analysis_df['diff2'].values < 0))
        concave_up_index = np.ravel(np.argwhere(analysis_df['diff2'].values > 0))

        if concave_down_index.size > 0:
            n_max = max_min(concave_down_index, analysis_df, 'max')
        if concave_up_index.size > 0:
            n_min = max_min(concave_up_index, analysis_df, 'min')

        n_max = n_max[n_max >= search_start]
        n_min = n_min[n_min >= search_start]   

        breath_data.update_one({'file_name': row['file_name'], 'breath_number': row['breath_number']+1}, {'$addToSet':{'characteristics.ie_min': n_min.tolist(), 'characteristics.ie_max': n_max.tolist()}})
        
        if n_max.size == 0 or n_min.size == 0:
            return False
        if n_max.size >= 1 and n_min.size >=1:
            try:
                for items in n_max:
                    if items + 4 < analysis_df['smooth'].shape[0] -1:
                        if analysis_df['smooth'].iloc[items] > analysis_df['smooth'].iloc[items-4:items+4].mean() *0.8:
                            return True
                    if analysis_df['smooth'].iloc[items] > analysis_df['smooth'].iloc[n_min[-1]] *0.9:
                        if items < n_min[-1]:
                            return True
                    if n_min[n_min < items].size > 0:
                        if analysis_df['smooth'].iloc[items] > analysis_df['smooth'].iloc[n_min[n_min < items][-1]] *0.5:
                            if np.absolute(items - n_min[n_min < items][-1]) < 3:
                                return True
            except:
                print('ie', n_max, n_min, analysis_df['smooth'].shape)
                print(sys.exc_info()[0])

        return False
    
    #find flow and pressure limitted breaths function
    def find_limited(row, p_or_f=p_or_f):
        analysis_df = make_analysis_df(row, p_or_f)

        n_max = np.array([])
        n_min = np.array([])

        exp_start = analysis_df[analysis_df.status == 1].shape[0]

        analysis_df['diff1'] = analysis_df['smooth'].diff()
        analysis_df['diff2'] = analysis_df['diff1'].diff()

        concave_down_index = np.ravel(np.argwhere(analysis_df['diff2'].values < 0))
        concave_up_index = np.ravel(np.argwhere(analysis_df['diff2'].values > 0))

        if concave_down_index.size > 0:
            n_max = max_min(concave_down_index, analysis_df, 'max')
        if concave_up_index.size > 0:
            n_min = max_min(concave_up_index, analysis_df, 'min')

        n_max = n_max[n_max <= exp_start+3]
        n_min = n_min[(n_min >= int(exp_start*0.1)) & (n_min <= exp_start-1)]

        breath_data.update_one({'file_name': row['file_name'], 'breath_number':row['breath_number']+1}, {'$addToSet':{'characteristics.'+p_or_f+'_min':n_min.tolist(), 'characteristics.'+p_or_f+'_max':n_max.tolist()}})
        if n_max.size == 0 or n_min.size == 0:
            return False

        if n_max[-1]/exp_start < 0.75:
            n_max = np.append(n_max, np.argmax(analysis_df[p_or_f].values[int(exp_start*0.75):])+int(exp_start*0.75))
        if n_min[0]/exp_start > 0.8:
            new_min = np.argmin(analysis_df[p_or_f].values[int(exp_start*0.25):int(exp_start*0.75)])+int(exp_start*0.25)
            n_min = np.append(np.array(new_min), n_min)

        ps = 0.2
        ptp = 1.025
        if (n_max.size >= 2) and (n_min.size >= 1):
            if n_max[0] < n_min[-1] < n_max[-1]:
                if (n_max[-1] - n_max[0])/exp_start > ps:
                    if analysis_df['smooth'].iloc[n_max[0]] > 5 and analysis_df['smooth'].iloc[n_max[-1]] > 5:
                        if (analysis_df[p_or_f].iloc[n_min[-1]] * ptp < analysis_df[p_or_f].iloc[n_max[0]]) or (analysis_df[p_or_f].iloc[n_min[-1]] * ptp < analysis_df[p_or_f].iloc[n_max[-1]]): 
                            if (analysis_df[p_or_f].iloc[n_min[-1]] <= analysis_df[p_or_f].iloc[n_max[0]]) and (analysis_df[p_or_f].iloc[n_min[-1]] <= analysis_df[p_or_f].iloc[n_max[-1]]):
                                return True
            if n_max[0] < n_min[0] < n_max[1]:
                if (n_max[1] - n_max[0])/exp_start > ps:
                    if analysis_df['smooth'].iloc[n_max[0]] > 5 and analysis_df['smooth'].iloc[n_max[1]] > 5:
                        if (analysis_df[p_or_f].iloc[n_min[0]] * ptp < analysis_df[p_or_f].iloc[n_max[0]]) or (analysis_df[p_or_f].iloc[n_min[0]] * ptp < analysis_df[p_or_f].iloc[n_max[1]]): 
                            if (analysis_df[p_or_f].iloc[n_min[0]] <= analysis_df[p_or_f].iloc[n_max[0]]) and (analysis_df[p_or_f].iloc[n_min[0]] <= analysis_df[p_or_f].iloc[n_max[1]]):
                                return True

        # Find Late Peaks
        if n_max.size == 1 and n_max/exp_start > 0.6 and p_or_f == 'flow':
            if np.less(analysis_df['diff1'].values[:exp_start], 7).all():
                return True

        # Find if No Sustained Rise
        if np.average(analysis_df[analysis_df.status ==1][p_or_f]) < np.average(analysis_df[analysis_df.status ==0][p_or_f]):
            return True 
        return False
    
    # Find PVT Breaths Function
    def find_pvt(row, p_or_f):
        analysis_df = make_analysis_df(row, p_or_f=p_or_f)
        len_at_end = analysis_df[analysis_df.status == 1].shape[0]
        status_df = analysis_df[len_at_end - 3:-1]

        if status_df.shape[0] < 5:
            print('nostatus')
            return False

        n_max = np.array([])
        n_min = np.array([])   

        try:
            init = int(status_df['smooth'].shape[0]*0)
            fin = int(status_df['smooth'].shape[0]*0.1+1)
            n_max = np.append(n_max, np.array([np.argmax(status_df['smooth'].values[init:fin])+init]))
            n_min = np.append(n_min, np.array([np.argmin(status_df['smooth'].values[init:fin])+init]))

            fin = int(status_df['smooth'].shape[0]*0.35+1)
            if n_min[0] +1 < status_df.shape[0]:
                n_max = np.append(n_max, np.array([np.argmax(status_df['smooth'].values[n_min[0]+1:fin])+n_min[0]+1]))
                n_min = np.append(n_min, np.array([np.argmin(status_df['smooth'].values[n_min[0]+1:fin])+n_min[0]+1]))

            if n_max[-1] < int(status_df.shape[0] *0.1):
                return False

            n_max = n_max + analysis_df[analysis_df.status == 1].shape[0] + init - 3
            n_min = n_min + analysis_df[analysis_df.status == 1].shape[0] + init - 3


        except:
            print(n_max, n_min, init, fin, status_df.shape)
            print('try', sys.exc_info()[0])

        try:
            if n_max[-1] >= n_min[-1] and n_max.size > 1:
                if n_max[-1] +3 < analysis_df.shape[0]:
                    init = n_max[-1]+3
                    fin = int(analysis_df[p_or_f].shape[0]-1)
                    n_min = np.append(n_min, np.array([np.argmin(analysis_df['smooth'].values[n_max[-1]:])+n_max[-1]+3]))
        except:
            print(n_max, n_min, init, fin, status_df.shape)
            print('try2', sys.exc_info()[0])

        n_max = np.unique(n_max)
        n_min = np.unique(n_min)

        breath_data.update_one({'file_name': row['file_name'], 'breath_number':row['breath_number']+1}, {'$addToSet':{'characteristics.pvt_min':n_min.tolist(), 'characteristics.pvt_max':n_max.tolist()}})
        if n_max.size == 0 or n_min.size == 0:
            return False

        try:
            # Find Early Peaks following first Min and Increased at Least 10% from Min
            if n_max.size >= 1 and n_min.size >=2:    
                if n_min[-2] < n_max[-1] < n_min[-1]:
                    if analysis_df[p_or_f].iloc[n_max[-1]] < 20:
                        if analysis_df[p_or_f].iloc[n_max[-1]]> analysis_df[p_or_f].iloc[n_min[-1]]:
                             if analysis_df[p_or_f].iloc[n_max[-1]] > analysis_df[p_or_f].iloc[n_min[-2]]:
                                return True
        except IndexError:
            n_min[-1] = analysis_df.shape[0] -1
            if n_max.size >= 1 and n_min.size >=2:    
                if n_min[-2] < n_max[-1] < n_min[-1]:
                    if analysis_df[p_or_f].iloc[n_max[-1]] < 20:
                        if analysis_df[p_or_f].iloc[n_max[-1]] *1.1 > analysis_df[p_or_f].iloc[n_min[-1]]:
                             if analysis_df[p_or_f].iloc[n_max[-1]] *1.1 > analysis_df[p_or_f].iloc[n_min[-2]]:
                                return True
        except:
            print(n_max, n_min, analysis_df.shape)
            print('pvt', sys.exc_info()[0])
            return False
        return False   
    
    # main program
    for criteria in patients:      
        # open data
        
        data = breath_data.find({'file_name':criteria})
        breath_df = pd.DataFrame(list(data))
        if breath_df.shape[0] != 0:
            # clean raw data
        
            breath_df = unpack(breath_df, unpack_col = ['characteristics', 'data_frame', 'vent_settings', 'analysis', 'validation'])

            breath_df.drop(['data_frame:DateTime', 'location'], inplace=True, axis=1)
            breath_df.sort(columns = 'start_time', inplace=True)
            breath_df.reset_index(inplace=True, drop=True)
            breath_df.reset_index(inplace=True, drop=False)
            breath_df.rename(columns={'index':'row_id'}, inplace=True)
            breath_df['characteristics:ds_vol'] = np.nan

            
            # Find DS Breaths
            @numbapro.vectorize([numbapro.boolean(numbapro.float64, numbapro.float64)])
            def ds(x, y):
                if (x < 0.75) & (y > 50): return True
                else: return False
            breath_df['analysis:ds'] = ds(breath_df['characteristics:exp_time'].values, breath_df['characteristics:end_insp_vol'].values)
            
            # Find Post_DS Breaths
            breath_df['characteristics:ds_vol'] = breath_df[(breath_df['analysis:ds'] == True)].apply(find_post_ds, axis=1)  
            try:
                breath_df.loc[(breath_df['analysis:pds'] != True), 'analysis:pds'] = False
            except KeyError:
                if 'analysis:pds' not in breath_df.columns:
                    breath_df['analysis:pds'] = np.nan
                pass
            
            # Find IE Breaths
            breath_df['analysis:ie'] = breath_df.apply(find_ie, p_or_f='flow', axis=1)

            # Find Flow Limited Breaths
            breath_df['analysis:fl'] = breath_df.apply(find_limited, p_or_f='flow', axis=1)

            # Find Pressure Limited Breaths
            breath_df['analysis:pl'] = breath_df.apply(find_limited, p_or_f='paw', axis=1)

            # Find PVT Breaths
            breath_df['analysis:pvt'] = breath_df.apply(find_pvt, p_or_f='flow', axis=1)

            # Summerize, Clean, and Save Findings to Database
            
                          
            summary_df = breath_df[['_id', 'analysis:ds', 'analysis:pds', 'analysis:ie', 'analysis:fl', 'analysis:pl', 'analysis:pvt', 'characteristics:ds_vol']]

            
            summary_df.rename(columns={'analysis:ds':'ds', 'analysis:pds':'pds', 'analysis:ie':'ie', 'analysis:fl':'fl', 
                                    'analysis:pl':'pl', 'analysis:pvt':'pvt'}, inplace=True)

            summary_dict = summary_df.to_dict(orient='records')

            print(criteria)
            for items in summary_dict:
                breath_data.update_one({'_id':items['_id']}, {'$set':{'analysis.ds':items['ds'],'analysis.pds':items['pds'],'analysis.ie':items['ie'],
                                                                      'analysis.fl':items['fl'],'analysis.pl':items['pl'],'analysis.pvt':items['pvt'], 
                                                                      'characteristics.ds_vol':items['characteristics:ds_vol']}})
    return patients

# Define Main Loop, Parallel By Patient

In [5]:
patients = breath_data.distinct('file_name')
#patients = test_data.distinct('file_name')
patients = list(patients)
print(patients)
#patients = patients[:]
chunks = [patients[x:x+12] for x in range(0, len(patients), 12)]

['P100\\Waveform 100-1.txt', 'P100\\Waveform 100-2.txt', 'P100\\Waveform 100-3.txt', 'P101\\hm_Waveform 14-04-22-18-38.txt', 'P101\\hm_Waveform 14-04-23-11-11.txt', 'P101\\hm_Waveform 14-04-23-12-25.txt', 'P101\\hm_Waveform 14-04-23-15-25.txt', 'P101\\hm_Waveform 14-04-24-09-43.txt', 'P101\\hm_Waveform 14-04-24-16-41.txt', 'P101\\hm_Waveform 14-04-25-07-51.txt', 'P101\\hm_Waveform 14-04-25-15-50.txt', 'P102\\hm_Waveform 14-05-10-16-57.txt', 'P102\\hm_Waveform 14-05-10-21-51.txt', 'P102\\hm_Waveform 14-05-10-22-03.txt', 'P103\\hm_Waveform 14-05-27-19-49.txt', 'P103\\hm_Waveform 14-05-28-04-56.txt', 'P103\\hm_Waveform 14-05-28-13-25.txt', 'P103\\hm_Waveform 14-05-28-17-55.txt', 'P103\\hm_Waveform 14-05-28-23-15.txt', 'P104\\hm_Waveform 05-07-01-20-55.txt', 'P105\\Waveform 14-09-12-1-11.txt', 'P105\\hm_Waveform 14-09-12-04-01.txt', 'P105\\hm_Waveform 14-09-14-04-21.txt', 'P105\\hm_Waveform 14-09-15-18-04.txt', 'P105\\hm_Waveform 14-09-16-06-30.txt', 'P105\\hm_Waveform 14-09-17-04-39.txt',

In [6]:
completed = [['P100\\Waveform 100-1.txt', 'P101\\hm_Waveform 14-04-22-18-38.txt', 'P100\\Waveform 100-2.txt', 'P102\\hm_Waveform 14-05-10-21-51.txt', 'P100\\Waveform 100-3.txt', 'P102\\hm_Waveform 14-05-10-16-57.txt', 'P122\\hm_Waveform 15-04-15-08-38.txt', 'P122\\hm_Waveform 15-04-14-22-57.txt', 'P122\\hm_Waveform 15-04-19-19-41.txt', 'P101\\hm_Waveform 14-04-25-15-50.txt', 'P101\\hm_Waveform 14-04-23-11-11.txt', 'P101\\hm_Waveform 14-04-23-12-25.txt'],
['P101\\hm_Waveform 14-04-24-09-43.txt', 'P101\\hm_Waveform 14-04-24-16-41.txt', 'P101\\hm_Waveform 14-04-25-07-51.txt', 'P122\\hm_Waveform 15-04-14-17-47.txt', 'P115\\hm_Waveform 15-01-29-17-19.txt', 'P113\\hm_Waveform 15-01-25-09-29.txt', 'P101\\hm_Waveform 14-04-23-15-25.txt', 'P122\\hm_Waveform 15-04-20-19-35.txt', 'P122\\hm_Waveform 15-04-22-06-47.txt', 'P102\\hm_Waveform 14-05-10-22-03.txt', 'P103\\hm_Waveform 14-05-27-19-49.txt', 'P122\\hm_Waveform 15-04-14-15-51.txt'],
['P122\\hm_Waveform 15-04-15-06-53.txt', 'P122\\hm_Waveform 15-04-22-12-57.txt', 'P103\\hm_Waveform 14-05-28-04-56.txt', 'P103\\hm_Waveform 14-05-28-13-25.txt', 'P103\\hm_Waveform 14-05-28-17-55.txt', 'P107\\hm_Waveform 14-10-03-18-09.txt', 'P104\\hm_Waveform 05-07-01-20-55.txt', 'P103\\hm_Waveform 14-05-28-23-15.txt', 'P105\\hm_Waveform 14-09-12-04-01.txt', 'P105\\hm_Waveform 14-09-14-04-21.txt', 'P105\\hm_Waveform 14-09-16-06-30.txt', 'P105\\hm_Waveform 14-09-17-04-39.txt'],
['P105\\hm_Waveform 14-09-15-18-04.txt', 'P106\\hm_Waveform 14-09-20-16-20.txt', 'P113\\hm_Waveform 15-01-24-09-47.txt', 'P113\\hm_Waveform 15-01-23-20-05.txt', 'P113\\hm_Waveform 15-01-24-12-42.txt', 'P113\\hm_Waveform 15-01-24-15-29.txt', 'P113\\hm_Waveform 15-01-24-20-44.txt', 'P113\\hm_Waveform 15-01-24-23-37.txt', 'P113\\hm_Waveform 15-01-25-04-53.txt', 'P114\\hm_Waveform 15-01-27-05-02.txt', 'P114\\hm_Waveform 15-01-27-14-44.txt', 'P115\\hm_Waveform 15-01-29-20-24.txt'],
['P110\\hm_Waveform 14-10-17-23-15.txt', 'P110\\hm_Waveform 14-10-19-06-54.txt', 'P113\\hm_Waveform 15-01-21-16-14.txt', 'P110\\hm_Waveform 14-10-17-17-07.txt', 'P105\\Waveform 14-09-12-1-11.txt', 'P106\\hm_Waveform 14-09-20-06-38.txt', 'P106\\hm_Waveform 14-09-20-07-28.txt', 'P106\\hm_Waveform 14-09-20-13-17.txt', 'P106\\hm_Waveform 14-09-19-21-41.txt', 'P106\\hm_Waveform 14-09-20-19-04.txt', 'P108\\hm_Waveform 14-10-07-10-33.txt', 'P113\\hm_Waveform 15-01-21-21-03.txt'],
['P113\\hm_Waveform 15-01-24-02-59.txt', 'P113\\hm_Waveform 15-01-24-06-15.txt', 'P107\\hm_Waveform 14-10-03-21-28.txt', 'P106\\hm_Waveform 14-09-20-22-39.txt', 'P113\\hm_Waveform 15-01-20-23-14.txt', 'P106\\hm_Waveform 14-09-21-07-26.txt', 'P106\\hm_Waveform 14-09-21-11-19.txt', 'P106\\hm_Waveform 14-09-21-13-10.txt', 'P109\\hm_Waveform 14-10-09-09-21.txt', 'P109\\hm_Waveform 14-10-09-11-28.txt', 'P109\\hm_Waveform 14-10-09-13-13.txt', 'P109\\hm_Waveform 14-10-09-16-58.txt'],
['P109\\hm_Waveform 14-10-09-14-25.txt', 'P109\\hm_Waveform 14-10-09-20-35.txt', 'P109\\hm_Waveform 14-10-09-21-19.txt', 'P109\\hm_Waveform 14-10-09-17-00.txt', 'P109\\hm_Waveform 14-10-09-22-51.txt', 'P109\\hm_Waveform 14-10-10-03-30.txt', 'P110\\hm_Waveform 14-10-17-04-51.txt', 'P113\\hm_Waveform 15-01-22-06-34.txt', 'P113\\hm_Waveform 15-01-22-09-44.txt', 'P113\\hm_Waveform 15-01-23-13-44.txt', 'P113\\hm_Waveform 15-01-23-10-17.txt', 'P113\\hm_Waveform 15-01-23-15-19.txt'],
['P113\\hm_Waveform 15-01-23-17-02.txt', 'P114\\hm_Waveform 15-01-28-05-15.txt', 'P120\\hm_Waveform 15-03-12-21-56.txt', 'P110\\hm_Waveform 14-10-18-21-34.txt', 'P115\\hm_Waveform 15-01-30-08-41.txt', 'P119\\hm_Waveform 15-03-06-08-07.txt', 'P121\\hm_Waveform 15-03-18-20-27.txt', 'P118\\hm_Waveform 15-02-24-11-24.txt', 'P110\\hm_Waveform 14-10-18-13-18.txt', 'P120\\hm_Waveform 15-03-11-13-24.txt', 'P120\\hm_Waveform 15-03-11-17-15.txt', 'P120\\hm_Waveform 15-03-12-05-43.txt'],
['P119\\hm_Waveform 15-03-07-20-41.txt', 'P110\\hm_Waveform 14-10-19-17-23.txt', 'P119\\hm_Waveform 15-03-11-06-41.txt', 'P110\\hm_Waveform 14-10-16-21-22.txt', 'P110\\hm_Waveform 14-10-20-06-27.txt', 'P119\\hm_Waveform 15-03-05-16-39.txt', 'P118\\hm_Waveform 15-02-25-11-31.txt', 'P109\\hm_Waveform 14-10-10-07-57.txt', 'P110\\hm_Waveform 14-10-16-15-39.txt', 'P110\\hm_Waveform 14-10-16-17-37.txt', 'P110\\hm_Waveform 14-10-17-07-25.txt', 'P110\\hm_Waveform 14-10-17-10-21.txt'],
['P110\\hm_Waveform 14-10-17-12-44.txt', 'P110\\hm_Waveform 14-10-17-18-49.txt', 'P110\\hm_Waveform 14-10-19-23-14.txt', 'P110\\hm_Waveform 14-10-21-07-31.txt', 'P110\\hm_Waveform 14-10-21-13-37.txt', 'P110\\hm_Waveform 14-10-20-12-17.txt', 'P112\\hm_Waveform 15-01-17-14-55.txt', 'P112\\hm_Waveform 15-01-17-18-39.txt', 'P112\\hm_Waveform 15-01-17-22-26.txt', 'P112\\hm_Waveform 15-01-18-04-40.txt', 'P112\\hm_Waveform 15-01-18-12-04.txt', 'P113\\hm_Waveform 15-01-20-16-22.txt'],
['P113\\hm_Waveform 15-01-20-21-13.txt', 'P113\\hm_Waveform 15-01-21-06-33.txt', 'P113\\hm_Waveform 15-01-21-13-17.txt', 'P113\\hm_Waveform 15-01-21-09-14.txt', 'P112\\hm_Waveform 15-01-18-11-41.txt', 'P119\\hm_Waveform 15-03-11-02-17.txt', 'P115\\hm_Waveform 15-01-30-16-35.txt', 'P119\\hm_Waveform 15-03-06-13-30.txt', 'P119\\hm_Waveform 15-03-05-10-46.txt', 'P110\\hm_Waveform 14-10-19-13-51.txt', 'P112\\hm_Waveform 15-01-18-09-40.txt', 'P112\\hm_Waveform 15-01-18-15-50.txt'],     
['P120\\hm_Waveform 15-03-13-08-10.txt', 'P119\\hm_Waveform 15-03-05-13-46.txt', 'P122\\hm_Waveform 15-04-18-16-18.txt', 'P119\\hm_Waveform 15-03-09-22-12.txt', 'P114\\hm_Waveform 15-01-27-09-13.txt', 'P121\\hm_Waveform 15-03-19-21-43.txt', 'P121\\hm_Waveform 15-03-18-17-59.txt', 'P117\\hm_Waveform 15-02-12-15-29.txt', 'P119\\hm_Waveform 15-03-06-16-11.txt', 'P120\\hm_Waveform 15-03-13-12-56.txt', 'P120\\hm_Waveform 15-03-12-09-08.txt', 'P118\\hm_Waveform 15-02-26-10-45.txt'],
['P115\\hm_Waveform 15-01-30-11-36.txt', 'P122\\hm_Waveform 15-04-17-11-59.txt', 'P121\\hm_Waveform 15-03-19-17-38.txt', 'P117\\hm_Waveform 15-02-10-12-31.txt', 'P122\\hm_Waveform 15-04-18-09-55.txt', 'P122\\hm_Waveform 15-04-16-13-33.txt', 'P120\\hm_Waveform 15-03-11-15-12.txt', 'P121\\hm_Waveform 15-03-20-08-39.txt', 'P116\\hm_Waveform 15-02-06-10-46.txt', 'P115\\hm_Waveform 15-01-30-06-56.txt', 'P115\\hm_Waveform 15-01-30-19-26.txt', 'P122\\hm_Waveform 15-04-19-13-18.txt'],
['P122\\hm_Waveform 15-04-16-10-37.txt', 'P122\\hm_Waveform 15-04-19-09-07.txt', 'P114\\hm_Waveform 15-01-28-12-13.txt', 'P122\\hm_Waveform 15-04-21-07-37.txt', 'P122\\hm_Waveform 15-04-15-16-50.txt', 'P117\\hm_Waveform 15-02-11-06-27.txt', 'P119\\hm_Waveform 15-03-09-15-40.txt', 'P119\\hm_Waveform 15-03-08-21-31.txt', 'P119\\hm_Waveform 15-03-08-15-40.txt', 'P122\\hm_Waveform 15-04-17-03-37.txt', 'P122\\hm_Waveform 15-04-20-07-18.txt', 'P117\\hm_Waveform 15-02-10-20-23.txt'],
['P119\\hm_Waveform 15-03-10-20-12.txt', 'P119\\hm_Waveform 15-03-10-08-21.txt', 'P122\\hm_Waveform 15-04-17-19-27.txt', 'P119\\hm_Waveform 15-03-09-09-04.txt', 'P121\\hm_Waveform 15-03-19-08-14.txt', 'P120\\hm_Waveform 15-03-12-14-47.txt', 'P122\\hm_Waveform 15-04-20-23-09.txt', 'P120\\hm_Waveform 15-03-12-12-10.txt', 'P122\\hm_Waveform 15-04-18-05-40.txt', 'P117\\hm_Waveform 15-02-10-22-16.txt', 'P117\\hm_Waveform 15-02-11-22-13.txt', 'P119\\hm_Waveform 15-03-08-10-20.txt'],
['P117\\hm_Waveform 15-02-11-12-41.txt', 'P114\\hm_Waveform 15-01-27-21-44.txt', 'P121\\hm_Waveform 15-03-18-14-21.txt', 'P122\\hm_Waveform 15-04-15-19-28.txt', 'P114\\hm_Waveform 15-01-28-08-36.txt', 'P117\\hm_Waveform 15-02-10-12-41.txt', 'P119\\hm_Waveform 15-03-10-15-20.txt', 'P114\\hm_Waveform 15-10-26-19-06.txt', 'P117\\hm_Waveform 15-02-12-06-19.txt', 'P122\\hm_Waveform 15-04-18-14-31.txt', 'P122\\hm_Waveform 15-04-19-17-05.txt', 'P120\\hm_Waveform 15-03-11-11-21.txt'],
['P120\\hm_Waveform 15-03-13-05-03.txt', 'P117\\hm_Waveform 15-02-13-06-31.txt', 'P118\\hm_Waveform 15-02-25-18-56.txt', 'P114\\hm_Waveform 15-01-26-23-22.txt', 'P119\\hm_Waveform 15-03-10-11-46.txt', 'P119\\hm_Waveform 15-03-10-22-14.txt', 'P115\\hm_Waveform 15-01-29-16-52.txt', 'P114\\hm_Waveform 15-01-27-18-25.txt', 'P116\\hm_Waveform 15-02-06-21-37.txt', 'P120\\hm_Waveform 15-03-12-18-42.txt', 'P122\\hm_Waveform 15-04-22-01-28.txt', 'P121\\hm_Waveform 15-03-19-14-17.txt'],
['P120\\hm_Waveform 15-03-11-19-34.txt', 'P119\\hm_Waveform 15-03-07-06-06.txt', 'P116\\hm_Waveform 15-02-06-15-00.txt', 'P122\\hm_Waveform 15-04-16-10-42.txt', 'P115\\hm_Waveform 15-01-30-15-32.txt', 'P115\\hm_Waveform 15-01-29-13-49.txt', 'P122\\hm_Waveform 15-04-15-12-02.txt', 'P117\\hm_Waveform 15-02-12-20-24.txt', 'P119\\hm_Waveform 15-03-06-22-02.txt', 'P122\\hm_Waveform 15-04-16-05-54.txt', 'P118\\hm_Waveform 15-02-26-07-15.txt', 'P122\\hm_Waveform 15-04-19-00-01.txt'],
['P117\\hm_Waveform 15-02-10-17-27.txt', 'P122\\hm_Waveform 15-04-16-22-10.txt', 'P121\\hm_Waveform 15-03-19-10-53.txt', 'P122\\hm_Waveform 15-04-21-15-17.txt', 'P122\\hm_Waveform 15-04-18-11-41.txt', 'P122\\hm_Waveform 15-04-21-20-28.txt', 'P119\\hm_Waveform 15-03-10-03-26.txt', 'P120\\hm_Waveform 15-03-11-13-20.txt'],
['P125\\hm_Waveform 15-07-13-03-49.txt', 'P125\\hm_Waveform 15-07-07-04-06.txt', 'P125\\hm_Waveform 15-07-14-08-08.txt', 'P129\\hm_Waveform 15-09-26-14-36.txt'],
['P125\\hm_Waveform 15-07-10-20-51.txt', 'P129\\hm_Waveform 15-09-24-04-32.txt', 'P125\\hm_Waveform 15-07-10-10-16.txt', 'P128\\hm_Waveform 15-09-09-15-29.txt', 'P129\\hm_Waveform 15-09-26-19-46.txt', 'P125\\hm_Waveform 15-07-13-01-22.txt', 'P125\\hm_Waveform 15-07-09-03-11.txt', 'P125\\hm_Waveform 15-07-07-18-42.txt'],
['P125\\hm_Waveform 15-07-11-22-44.txt', 'P130\\hm_Waveform 15-10-13-08-14.txt', 'P125\\hm_Waveform 15-07-08-08-49.txt', 'P127\\hm_Waveform 15-08-26-04-16.txt', 'P125\\hm_Waveform 15-07-07-21-11.txt', 'P129\\hm_Waveform 15-09-28-09-08.txt', 'P129\\hm_Waveform 15-09-23-20-08.txt', 'P124\\hm_Waveform 15-06-04-13-26.txt', 'P125\\hm_Waveform 15-07-06-01-04.txt', 'P125\\hm_Waveform 15-07-14-06-11.txt', 'P125\\hm_Waveform 15-07-06-06-52.txt', 'P129\\hm_Waveform 15-09-25-07-10.txt'],
['P125\\hm_Waveform 15-07-14-18-59.txt', 'P129\\hm_Waveform 15-09-26-11-14.txt', 'P124\\hm_Waveform 15-06-04-23-24.txt', 'P124\\hm_Waveform 15-06-03-07-01.txt', 'P125\\hm_Waveform 15-07-08-01-13.txt', 'P126\\hm_Waveform 15-07-17-13-19.txt', 'P126\\hm_Waveform 15-07-17-07-06.txt', 'P125\\hm_Waveform 15-07-11-01-34.txt', 'P124\\hm_Waveform 15-06-05-08-11.txt', 'P129\\hm_Waveform 15-09-27-17-28.txt', 'P124\\hm_Waveform 15-06-04-09-05.txt', 'P125\\hm_Waveform 15-07-06-11-04.txt'],
            ['P125\\hm_Waveform 15-07-08-14-17.txt', 'P124\\hm_Waveform 15-06-02-16-40.txt', 'P125\\hm_Waveform 15-07-13-01-12.txt', 'P125\\hm_Waveform 15-07-13-22-46.txt', 'P127\\hm_Waveform 15-08-28-12-31.txt', 'P129\\hm_Waveform 15-09-24-11-52.txt', 'P130\\hm_Waveform 15-10-13-01-14.txt', 'P124\\hm_Waveform 15-06-03-13-36.txt', 'P129\\hm_Waveform 15-09-28-15-07.txt', 'P125\\hm_Waveform 15-07-06-04-08.txt', 'P125\\hm_Waveform 15-07-07-12-55.txt', 'P127\\hm_Waveform 15-08-28-04-09.txt'],
            ['P125\\hm_Waveform 15-07-08-19-43.txt', 'P124\\hm_Waveform 15-06-02-20-12.txt', 'P129\\hm_Waveform 15-09-25-13-35.txt', 'P125\\hm_Waveform 15-07-12-02-55.txt', 'P129\\hm_Waveform 15-09-24-13-48.txt', 'P127\\hm_Waveform 15-08-27-12-26.txt', 'P126\\hm_Waveform 15-07-16-23-03.txt', 'P130\\hm_Waveform 15-10-13-12-01.txt', 'P126\\hm_Waveform 15-07-16-19-10.txt', 'P127\\hm_Waveform 15-08-25-15-58.txt', 'P125\\hm_Waveform 15-07-14-00-49.txt']]

In [7]:
import itertools
total = set(patients)
done = set(itertools.chain.from_iterable(completed))
to_do = list(total-done)
print(to_do)

['P125\\hm_Waveform 15-07-13-03-49.txt', 'P125\\hm_Waveform 15-07-07-04-06.txt', 'P125\\hm_Waveform 15-07-14-08-08.txt', 'P129\\hm_Waveform 15-09-26-14-36.txt', 'P125\\hm_Waveform 15-07-10-20-51.txt', 'P129\\hm_Waveform 15-09-24-04-32.txt', 'P125\\hm_Waveform 15-07-10-10-16.txt', 'P128\\hm_Waveform 15-09-09-15-29.txt', 'P129\\hm_Waveform 15-09-26-19-46.txt', 'P125\\hm_Waveform 15-07-13-01-22.txt', 'P125\\hm_Waveform 15-07-09-03-11.txt', 'P125\\hm_Waveform 15-07-07-18-42.txt', 'P125\\hm_Waveform 15-07-11-22-44.txt', 'P130\\hm_Waveform 15-10-13-08-14.txt', 'P125\\hm_Waveform 15-07-08-08-49.txt', 'P127\\hm_Waveform 15-08-26-04-16.txt', 'P125\\hm_Waveform 15-07-07-21-11.txt', 'P129\\hm_Waveform 15-09-28-09-08.txt', 'P129\\hm_Waveform 15-09-23-20-08.txt', 'P124\\hm_Waveform 15-06-04-13-26.txt', 'P125\\hm_Waveform 15-07-06-01-04.txt', 'P125\\hm_Waveform 15-07-14-06-11.txt', 'P125\\hm_Waveform 15-07-06-06-52.txt', 'P129\\hm_Waveform 15-09-25-07-10.txt', 'P125\\hm_Waveform 15-07-14-18-59.txt',

In [8]:
chunks = [to_do[x:x+12] for x in range(0, len(to_do), 12)]

In [9]:
print(chunks)

[['P125\\hm_Waveform 15-07-13-03-49.txt', 'P125\\hm_Waveform 15-07-07-04-06.txt', 'P125\\hm_Waveform 15-07-14-08-08.txt', 'P129\\hm_Waveform 15-09-26-14-36.txt', 'P125\\hm_Waveform 15-07-10-20-51.txt', 'P129\\hm_Waveform 15-09-24-04-32.txt', 'P125\\hm_Waveform 15-07-10-10-16.txt', 'P128\\hm_Waveform 15-09-09-15-29.txt', 'P129\\hm_Waveform 15-09-26-19-46.txt', 'P125\\hm_Waveform 15-07-13-01-22.txt', 'P125\\hm_Waveform 15-07-09-03-11.txt', 'P125\\hm_Waveform 15-07-07-18-42.txt'], ['P125\\hm_Waveform 15-07-11-22-44.txt', 'P130\\hm_Waveform 15-10-13-08-14.txt', 'P125\\hm_Waveform 15-07-08-08-49.txt', 'P127\\hm_Waveform 15-08-26-04-16.txt', 'P125\\hm_Waveform 15-07-07-21-11.txt', 'P129\\hm_Waveform 15-09-28-09-08.txt', 'P129\\hm_Waveform 15-09-23-20-08.txt', 'P124\\hm_Waveform 15-06-04-13-26.txt', 'P125\\hm_Waveform 15-07-06-01-04.txt', 'P125\\hm_Waveform 15-07-14-06-11.txt', 'P125\\hm_Waveform 15-07-06-06-52.txt', 'P129\\hm_Waveform 15-09-25-07-10.txt'], ['P125\\hm_Waveform 15-07-14-18-59.

In [60]:
for items in chunks:
    print(parallel_dyssynchrony_finder(items))

['P125\\hm_Waveform 15-07-13-03-49.txt', 'P125\\hm_Waveform 15-07-07-04-06.txt', 'P125\\hm_Waveform 15-07-14-08-08.txt', 'P129\\hm_Waveform 15-09-26-14-36.txt', 'P125\\hm_Waveform 15-07-10-20-51.txt', 'P129\\hm_Waveform 15-09-24-04-32.txt', 'P125\\hm_Waveform 15-07-10-10-16.txt', 'P128\\hm_Waveform 15-09-09-15-29.txt', 'P129\\hm_Waveform 15-09-26-19-46.txt', 'P125\\hm_Waveform 15-07-13-01-22.txt', 'P125\\hm_Waveform 15-07-09-03-11.txt', 'P125\\hm_Waveform 15-07-07-18-42.txt']
['P125\\hm_Waveform 15-07-11-22-44.txt', 'P130\\hm_Waveform 15-10-13-08-14.txt', 'P125\\hm_Waveform 15-07-08-08-49.txt', 'P127\\hm_Waveform 15-08-26-04-16.txt', 'P125\\hm_Waveform 15-07-07-21-11.txt', 'P129\\hm_Waveform 15-09-28-09-08.txt', 'P129\\hm_Waveform 15-09-23-20-08.txt', 'P124\\hm_Waveform 15-06-04-13-26.txt', 'P125\\hm_Waveform 15-07-06-01-04.txt', 'P125\\hm_Waveform 15-07-14-06-11.txt', 'P125\\hm_Waveform 15-07-06-06-52.txt', 'P129\\hm_Waveform 15-09-25-07-10.txt']
['P125\\hm_Waveform 15-07-14-18-59.txt